## Import Library

In [1]:
import pandas as pd
import numpy as np

import pickle, gzip, pickletools

## Import Books

In [11]:
books = pd.read_csv("./input/books.csv")
books['JUDUL BUKU']=books['JUDUL BUKU'].str.strip()
books

,ID,TANGGAL MASUK,NO INVENTARIS,PENYUSUN,JUDUL BUKU,PENERBIT,KOTA,TAHUN TERBIT,CALL NUMBER KLASIFIKASI,CALL NUMBER PENGARANG,CALL NUMBER KLASIFIKASI JUDUL,SUBYEK,ASAL,EKS,KET
0,1,19-4-2023,01/SMPN3 KDR/2023,RPUL BUANA RAYA,RANGKUMAN PENGETAHUAN UMUM,RPUL BUANA RAYA,JAKARTA,2014,000,BUA,RPA,KARYA UMUM,DAK203,1,BAIK
1,2,19-4-2023,02/SMPN3 KDR/2023,TIM CBM,MENJELAJAH SUNGAI NIL,NUANSA CITRA DRAFIKA,JAKARTA,2008,001,TIM,MEN,ILMU PENGETAHUAN UMUM,DAK203,1,BAIK
2,3,19-4-2023,03/SMPN3 KDR/2023,HENDY HERMAWAN,GURU MENULIS,CITRA PRAYA,BANDUNG,2006,070,HEN,GUR,JURNALISM,DAK203,1,BAIK
3,4,19-4-2023,04-05/SMPN3 KDR/2023,SUHARNO WIDI NUGROHO ST,MENGENAL LEMBAR KERJA MS EKCEL,BSD MIPA,JAKARTA,2010,004,SUH,MEN,PENGOLAHAN DATA ILMU KOMPUTER,DAK203,2,BAIK
4,5,19-4-2023,06-07/SMPN3 KDR/2023,SANYO SAPUTRA,MENGGAMBAR DENGAN MICROSOFT PAINT,EXACT GANECA,JAKARTA,2007,004,SAN,MEN,PENGOLAHAN DATA ILMU KOMPUTER,DAK203,2,BAIK
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1314,1315,24-6-2023,2438-2440/SMPN3 KDR/2023,ALANNA FARDIANI,AYO MENGUKUR KELAJUAN,GRAHA ILMU MULIA,SURABAYA,2009,"516,9",ALA,AYO,ILMU UKUR DESKRIPTIF,DAK203,3,BAIK
1315,1316,24-6-2023,2441-2442/SMPN3 KDR/2023,RUMIDA LUMBAN RAJA,BAWANG MERAH,PANCA ANUGRAH SAKTI,TANGERANG,2010,581,RUM,BAW,BOTANI,DAK203,2,BAIK
1316,1317,24-6-2023,2443-2445/SMPN3 KDR/2023,ALANNA FARDIANI,AYO MENGUKUR BERAT,GRAHA ILMU MULIA,SURABAYA,2009,"516,9",ALA,AYO,ILMU UKUR DESKRIPTIF,DAK203,3,BAIK
1317,1318,24-6-2023,2446-2448/SMPN3 KDR/2023,DIDIK DJUNAEDI,MENGENAL BANGUN DAN BELAJAR PECAHAN,GADING INTI PRIMA,JAKARTA,2010,513,DID,MEN,ILMU HITUNG,DAK203,3,BAIK


## Import Model

In [12]:
with gzip.open("svd_model.h5", 'rb') as f:
    p = pickle.Unpickler(f)
    model = p.load()

In [13]:
model

In [14]:
pd.DataFrame(model.qi)

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,-0.043023,-0.319651,-0.066558,-0.077602,-0.150253,0.297176,-0.125661,-0.001561,0.289335,0.061583,...,0.029865,0.072919,-0.043275,0.177479,0.191562,-0.208198,0.024303,0.459184,-0.068537,0.029893
1,-0.214452,-0.101327,0.086615,0.048393,-0.054353,0.206596,-0.007356,0.046981,0.159609,-0.095499,...,0.276162,0.119729,0.019788,-0.188448,0.072627,0.011838,-0.143431,-0.059901,0.109201,-0.108005
2,0.225720,-0.167290,0.222587,-0.191411,0.314358,0.184888,0.353982,0.299889,-0.027567,0.025194,...,0.212401,-0.037135,0.049537,-0.044625,0.074143,-0.299344,-0.149682,0.046827,0.242303,0.112694
3,0.124207,-0.019404,-0.065053,0.078118,-0.147969,0.006256,-0.203950,-0.257661,-0.279606,-0.088558,...,-0.267170,0.266495,-0.116434,0.021859,-0.132600,-0.211786,-0.188322,0.407447,-0.023852,0.065913
4,-0.060680,-0.279450,-0.139020,-0.044696,0.081966,0.160476,0.039470,-0.079144,0.120323,0.240208,...,0.162350,-0.094781,-0.142539,-0.001717,0.187580,-0.155628,-0.253873,0.090758,0.097053,-0.016212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
881,0.168362,0.047393,0.205747,0.061471,0.003274,-0.104947,0.275797,-0.211896,0.062653,-0.039534,...,0.258914,0.045883,-0.018999,0.173652,0.035382,0.120457,0.141245,0.047813,-0.033542,-0.094441
882,-0.078118,-0.055733,0.177521,0.153407,-0.073289,-0.354206,-0.058046,-0.092125,-0.096062,0.049735,...,-0.143682,0.056484,0.060044,0.212731,0.187492,0.047430,0.360591,0.139575,0.030008,-0.231229
883,0.137348,0.281783,0.245553,-0.009197,0.225086,-0.208190,-0.006119,-0.038175,0.139033,-0.118896,...,0.214449,-0.040270,-0.212041,0.079835,0.049267,0.195690,-0.047314,0.146207,0.164586,-0.083914
884,0.080326,-0.145351,0.353249,0.004022,0.151557,-0.088666,0.051644,-0.005518,0.213196,-0.276439,...,-0.090508,0.117698,-0.025483,-0.083395,0.053403,0.174225,-0.196947,-0.064747,-0.174062,-0.138868


In [15]:
model.qi.shape

(886, 100)

## Import Cosine Distance

In [16]:
from scipy.spatial.distance import cosine as cosine_distance

## Get Prediction

### Get Book Latent Features

In [17]:
def get_vector(raw_id: int, trained_model=model) -> np.array:
    """Returns the latent features of a movie in the form of a numpy array"""
    book_row_idx = trained_model.trainset._raw2inner_id_items[raw_id]
    return trained_model.qi[book_row_idx]

### Get Book Recommendation

In [23]:
def get_recs(liked_book_title: str, model=model):
    # try:
        """Returns the top 25 most similar books to a specified book
        
        This function iterates over every possible book in dataset and calculates
        distance between `book_title` vector and that book's vector.
        """
        # Get book id from data by book title
        liked_raw_id = books[books['JUDUL BUKU']==liked_book_title]["ID"].item()

        # Get the first book vector
        book_vector: np.array = get_vector(liked_raw_id, model)
        similarity_table = []
        
        # Iterate over every possible book and calculate similarity
        for other_raw_id in model.trainset._raw2inner_id_items.keys():
            other_book_vector = get_vector(other_raw_id, model)
            
            # Get the second book vector, and calculate distance
            similarity_score = cosine_distance(other_book_vector, book_vector)
            recommended_book = books[books['ID']==other_raw_id]["JUDUL BUKU"].item()

            if similarity_score != 0:
                similarity_table.append((similarity_score, recommended_book))

        recs = pd.DataFrame(sorted(similarity_table), columns=["vector cosine distance", "JUDUL BUKU"])
        # sort books by ascending similarity
        return recs.head(25)
    # Exception for if there isnt enough info about the book
    # except:
    #     print("Not enough info about book")

In [26]:
get_recs("MICROSOFT OFFICE")

,vector cosine distance,JUDUL BUKU
0,0.601128,MENGENAL PIRANTI KOMPUTER PRIBADI
1,0.605769,PENDIDIKAN KEWIRAUSAHAAN
2,0.613036,MENGOPRASIKAN KOMPUTER
3,0.614054,MENGOLAH GAMBAR SEDERHANA DENGAN PHOTOSHOP
4,0.615096,FISIKA CERIA SERI KALOR
5,0.618759,PHOTOSHOP BIKIN FOTO MANIPULATIF NAN KREATIF
6,0.640530,KUMPULAN PERI BAHASA
7,0.662830,MEMBUAT DOKUMEN SEDERHANA MENGGUNAKAN WORD 2007
8,0.683157,TERAMPIL BERMUSIK
9,0.685117,WIKIPEDIA UNTUK GURU


In [27]:
get_recs("ASYIK BELAJAR MATEMATIKA")

,vector cosine distance,JUDUL BUKU
0,0.475511,KAMUS ISTILAH BERGAMBAR MATEMATIKA
1,0.554274,FISIKA CERIA SERI KALOR
2,0.556175,MENGENAL BILANGAN
3,0.573828,PINTAR FISIKA
4,0.592052,FISIKA CERIA SERI OPTIK GEOMETRI
5,0.619279,MARI BEREKSPERIMEN DENGAN SAINS
6,0.630037,MENITI HARI DEPAN
7,0.644489,SAINS UNTUK PEMULA 5 MARI BERMAIN BUNYI
8,0.645726,MANAJEMEN KELAS UNTUK MENCIPTAKAN IKLIM BELAJAR
9,0.655028,ENSIKLOPEDIA PELAJAR TEMATIK


In [28]:
get_recs("ENGLISH EVERYWHERE")

,vector cosine distance,JUDUL BUKU
0,0.427852,BERBAHASA INGGRIS YANG BAIK DAN BENAR
1,0.449511,THE GOATS SECRET AND OTHER STORIES
2,0.450919,BELAJAR BAHASA INGGRIS MELALUI PERMAINAN
3,0.454089,LEST UNDERSTAND ENGLISH
4,0.495727,ENGLISH COMPETENCY READING COMREHENSION
5,0.511771,THE FISH AND THE TORTOIS
6,0.554669,KAMUS IDEAL
7,0.565211,SINONIM DAN ANTONIM DALAM BAHASA INGGRIS
8,0.582704,DON’T JUDGE A MAN BY HIS FACE
9,0.586974,PANDUAN PRAKTIS BERBUSANA


In [35]:
get_recs("YAJNA SESA")

,vector cosine distance,JUDUL BUKU
0,0.492275,SISYA SESANA
1,0.520930,MANAJEMEN MUTU BERBASIS SEKOLAH
2,0.617217,TIPS DAN TRIK TERSEMBUNYI MICROSOFT OFFICE 2013
3,0.635156,SATUA BALI KAMBING TAKUTIN MACAN
4,0.660494,TATA CARA MELAMAR PEKERJAAN
5,0.662155,LEGENDA TIMUN MAS
6,0.688124,SUKSES BERBISNIS DENGAN BANGSA CINA
7,0.701497,RISET KEHUMASAN
8,0.702958,HUTAN
9,0.703776,LANGKAH SUKSES BELAJAR BAHASA INGGRIS 1
